<a href="https://colab.research.google.com/github/rswijesena/AI/blob/main/Advance_OpenAI_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install panda
!pip install pip install langchain

In [43]:
import openai
from openai import OpenAI

openAIKey="OPEN_API_KEY"

client = OpenAI(api_key=openAIKey)

function_description = [
    {
        "name": "get_flight_info",
        "description": "Get the next flight from delhi to mumbai",
        "parameters": {
            "type": "object",
            "properties": {
                "from": {"type": "string", "description": "The depature airport e.g DEL"},
                "to": {"type": "string", "description": "The destination airport e.g SYD"},
                "date": {"type": "string"},
            },
            "required": ["from", "to"],
        },
    }
]

In [44]:
user_prompt = "When's the next flight from Colombo to Sydney? "

In [45]:
respones2

ChatCompletion(id='chatcmpl-9Oim8d0N18nRer2PNTYmaxNn9xYGl', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"from":"CMB","to":"SYD"}', name='get_flight_info'), tool_calls=None))], created=1715679016, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=89, total_tokens=110))

In [46]:
import json
from datetime import datetime, timedelta
def get_flight_info(from_airport, to_airport):
  """Get Flight information between two airports"""
  #Example out put

  flight_info = {
      "from_airport": from_airport,
      "to_airport": to_airport,
      "date": str(datetime.now() + timedelta(hours=2)),
      "airline" : "Qantas",
      "flight_number": "QF466"
  }
  return json.dumps(flight_info)

In [47]:
origin = json.loads(respones2.choices[0].message.function_call.arguments).get("from")
destination = json.loads(respones2.choices[0].message.function_call.arguments).get("to")

origin,destination

('CMB', 'SYD')

In [48]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": user_prompt
        },
        {
            "role": "function",
            "name": "get_flight_info",
            "content": get_flight_info(origin, destination)
        }
    ],
    #Add function calling
    functions=function_description,
    function_call="auto" # specify the function call

)

response3.choices[0].message.content

'The next flight from Colombo (CMB) to Sydney (SYD) is on May 14, 2024. The flight is operated by Qantas with the flight number QF466.'